In [21]:
'''
README:

This code uses schedule GTFS to calculate geometric features for stops and shapes before using those
data to create interpolated points '''


# Create the trips_shaped layer

import os
import csv

def create_trips_shaped(input_folder, workspace):
    shapes_file = os.path.join(input_folder, 'shapes.txt')
    polyline_list = []
    previous_id = None
    array = arcpy.Array()

    with arcpy.da.SearchCursor(shapes_file, ['shape_id', 'shape_pt_lat', 'shape_pt_lon'], sql_clause=(None, 'ORDER BY shape_id')) as cursor:
        for row in cursor:
            unique_id, latitude, longitude = row
            if unique_id != previous_id:
                if array.count > 1:
                    polyline = arcpy.Polyline(array)
                    polyline_list.append(polyline)
                array.removeAll()

            point = arcpy.Point(longitude, latitude)
            array.add(point)
            previous_id = unique_id

    # Add the last polyline
    if array.count > 1:
        polyline = arcpy.Polyline(array)
        polyline_list.append(polyline)

    # Now, the `polyline_list` contains all the individual polyline objects
    # for index, polyline in enumerate(polyline_list):
        # print(f'Polyline {index + 1}: {polyline.JSON}')
        
    trips_file = os.path.join(input_folder, 'trips.txt')
    output_fc_name = 'Trips_Shaped_20240323'
    spatial_reference = arcpy.SpatialReference(4326)
    arcpy.CreateFeatureclass_management(workspace, output_fc_name, 'POLYLINE', spatial_reference=spatial_reference)
    fields = ['route_id', 'trip_id', 'trip_headsign', 'direction', 'shape_id']
    iterator = -1
    existing_shape_id = ''


    for field in fields:
        arcpy.AddField_management(os.path.join(workspace, output_fc_name), field, 'TEXT')

    with open(trips_file, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        sorted_rows = sorted(csvreader, key=lambda row: row['shape_id'])

        for row in sorted_rows:
            row_shape_id = row['shape_id']
            if existing_shape_id != row_shape_id:
                iterator = iterator + 1
                existing_shape_id = row_shape_id
            polyline = polyline_list[iterator]
            with arcpy.da.InsertCursor(os.path.join(workspace, output_fc_name), fields + ['SHAPE@']) as cursor:
                route_id = row['route_id']
                trip_id = row['trip_id']
                trip_headsign = row['trip_headsign']
                direction = row['direction']
                shape_id = row['shape_id']
                cursor.insertRow([route_id, trip_id, trip_headsign, direction, shape_id, polyline])

    print("Process completed successfully.")  
    
create_trips_shaped(r'C:\Users\BRAUE189\Downloads\gtfs20240323', r'C:\Users\BRAUE189\Documents\ArcGIS\Projects\GIS5572\Default.gdb')    

Process completed successfully.


In [2]:
# Create the stops_geo layer

import os
import csv
from tqdm import tqdm

def create_stops_geo(input_folder, spatial_ref):
    stop_times = os.path.join(input_folder, 'stop_times.txt')
    stops = os.path.join(input_folder, 'stops.txt')
    stop_times_geo = os.path.join(input_folder, 'stops_times_geo.txt')
    unique_identifier_field = 'stop_id'
    data_dict_file2 = {}
    merged_data = []

    with open(stops, 'r', newline='') as file2:
        csvreader = csv.DictReader(file2)
        for row in tqdm(csvreader, desc='Processing step1'):
            stop_id = row[unique_identifier_field]
            data_dict_file2[stop_id] = {key: row[key] for key in row if key != unique_identifier_field}

    with open(stop_times, 'r', newline='') as file1:
        csvreader = csv.DictReader(file1)
        for row in tqdm(csvreader, desc='Processing step2'):
            stop_id = row[unique_identifier_field]
            data_from_file2 = data_dict_file2.get(stop_id, {})
            row.update(data_from_file2)
            merged_data.append(row)

    with open(stop_times_geo, 'w', newline='') as merged_file:
        fieldnames = merged_data[0].keys() 
        writer = csv.DictWriter(merged_file, fieldnames = fieldnames)
        writer.writeheader()
        for row in tqdm(merged_data, desc='Processing step3'):
            writer.writerow(row)
        
    output_coordinate_system = arcpy.SpatialReference(spatial_ref)
    out_feature_class = 'Stops_Geo_20240323'
    output_projected_fc = 'Stops_Geo_Projected_20240323'
    arcpy.management.XYTableToPoint(stop_times_geo, out_feature_class, 'stop_lon', 'stop_lat')
    arcpy.management.Project(out_feature_class, output_projected_fc, output_coordinate_system)
    
    print("Process completed successfully.") 
    
create_stops_geo(r'C:\Users\BRAUE189\Downloads\gtfs20240323', 26915)

Processing step1: 9031it [00:00, 125719.01it/s]
Processing step2: 1576597it [00:08, 196304.03it/s]﻿
Processing step3: 100%|██████████| 1576597/1576597 [00:17<00:00, 90748.26it/s]﻿


Process completed successfully.


In [4]:
import datetime
from tqdm import tqdm

def snap_and_interpolate(workspace, input_line_fc, input_points_fc):
    # Set up workspace and input data
    arcpy.env.workspace = workspace
    output_points_fc = 'new_points'

    # Create a new feature class for the interpolated points
    output_points_path = arcpy.CreateFeatureclass_management(arcpy.env.workspace, output_points_fc, 'POINT', spatial_reference=input_line_fc)

    # Add fields to the output feature class
    arcpy.AddField_management(output_points_path, 'time', 'TIMEONLY')
    arcpy.AddField_management(output_points_path, 'trip_id', 'TEXT')


    # Collect rows to insert
    rows_to_insert = []

    trip_ids = set(row[0] for row in arcpy.da.SearchCursor(input_line_fc, 'trip_id'))

    for trip_id in tqdm(trip_ids, desc='Processing trips'):
        with arcpy.da.SearchCursor(input_line_fc, ['SHAPE@'], f"trip_id = '{trip_id}'") as cursor1:
            for row1 in cursor1:
                line = row1[0]
                prev_arrival_time = None
                prev_departure_time = None
                prev_point = None

                with arcpy.da.SearchCursor(input_points_fc, ['SHAPE@', 'arrival_time', 'departure_time', 'trip_id'], f"trip_id = '{trip_id}'", sql_clause=(None, "ORDER BY stop_sequence")) as cursor:
                    for row in cursor:
                        point = row[0]
                        arrival_time = row[1]
                        departure_time = row[2]
                        trip_id = row[3]

                        if prev_arrival_time is not None and prev_departure_time is not None:
                            # Calculate time difference using timedelta
                            arrival_datetime = datetime.datetime.combine(datetime.date.today(), arrival_time)
                            departure_datetime = datetime.datetime.combine(datetime.date.today(), prev_departure_time)

                            # Calculate time difference in seconds
                            time_diff = int((arrival_datetime - departure_datetime).total_seconds() / 60)   

                            if time_diff > 1:
                                sub_line = line.segmentAlongLine(line.measureOnLine(prev_point), line.measureOnLine(point))

                                for t in range(1, time_diff):
                                    fraction = t / time_diff
                                    new_point = sub_line.positionAlongLine(fraction, True)
                                    rows_to_insert.append([new_point, departure_datetime + datetime.timedelta(seconds = t * 60), trip_id])

                        prev_arrival_time = arrival_time
                        prev_departure_time = departure_time
                        prev_point = point

    # Insert all rows at once
    with arcpy.da.InsertCursor(output_points_fc, ['SHAPE@', 'time', 'trip_id']) as insert_cursor:
        for row in rows_to_insert:
            insert_cursor.insertRow(row)

    field_mapping = 'time "time" true true false 8 TimeOnly 0 0,First,#,Stops_Geo_Projected,departure_time,-1,-1;trip_id "trip_id" true true false 255 Text 0 0,First,#,Stops_Geo_Projected,trip_id,0,7999'
    arcpy.management.Append(input_points_fc, output_points_fc, 'NO_TEST', field_mapping)
    
snap_and_interpolate(r'C:\Users\BRAUE189\Documents\ArcGIS\Projects\GIS5572\Default.gdb', 'Trips_Shaped', 'Stops_Geo_Projected')

Processing trips: 100%|██████████| 29587/29587 [5:09:47<00:00,  1.59it/s]  ﻿


In [41]:
from datetime import datetime, timedelta
from tqdm import tqdm

def time_diff_in_minutes(time1, time2):
    total_seconds1 = time1.hour * 3600 + time1.minute * 60 + time1.second
    total_seconds2 = time2.hour * 3600 + time2.minute * 60 + time2.second
    diff_seconds = abs(total_seconds1 - total_seconds2)
    diff_minutes = diff_seconds // 60
    return diff_minutes

def calculate_delay(live_workspace, table_name, local_workspace, start_time):
    
    current_time = start_time
    end_time = start_time + timedelta(days = 7)
    
    while current_time < end_time:
        # Create datetime objects for analysis and convert from utc to local time and query live bus data
        utc_time = current_time + timedelta(hours = 5)
        time_plus_one = utc_time + timedelta(minutes = 1)
        
        # Start to iterate through to select only scheduled points from that time and day
        scheduled_bus_points = 'new_points'
        delay_day = f'Delay_{current_time.strftime("%A")}'
        day_status = f'{current_time.strftime("%A")}_Status'
        current_minute = current_time.time()
        
        # Create a temporary feature class to hold the points from the array and clear selection
        query = f"SELECT ST_SetSRID(ST_MakePoint(longitude, latitude), 4326) as geom, timestamp, trip_id FROM {table_name} WHERE timestamp > timestamp '{utc_time}' And timestamp < timestamp '{time_plus_one}'"
        layer_name = "temp_live_points"
        arcpy.MakeQueryLayer_management(live_workspace, layer_name, query, "OBJECTID", "POINT", "4326")
        near_trip_ids_time = {}
        
        # Pull the correct trip_id and time 
        arcpy.env.workspace = local_workspace
        with arcpy.da.SearchCursor('temp_live_points', ['trip_id', 'SHAPE@']) as cursor:
            for row in tqdm(cursor, desc='Processing points'):
                trip_id = row[0]
                arcpy.management.SelectLayerByAttribute(scheduled_bus_points, "NEW_SELECTION", f"trip_id = '{trip_id}'")

                # Run the nearest table using the row and the scheduled data and copy the corresponding trip_id and time into the dictionary
                arcpy.analysis.GenerateNearTable(row[1], scheduled_bus_points, 'live_points_near_table', search_radius = "500 Meters", closest="CLOSEST")

                # Get the nearest point's trip_id and time
                with arcpy.da.SearchCursor('live_points_near_table', ['NEAR_FID']) as near_cursor:
                    for near_row in near_cursor:
                        near_id = near_row[0]
                        with arcpy.da.SearchCursor(scheduled_bus_points, ['trip_id', 'time'], f"OBJECTID = {near_id}") as trip_cursor:
                            for trip_row in trip_cursor:
                                near_trip_ids_time[trip_row[0]] = trip_row[1]
        
        arcpy.management.SelectLayerByAttribute(scheduled_bus_points, "CLEAR_SELECTION")
        arcpy.management.Delete('live_points_near_table')
        arcpy.management.Delete('temp_live_points')
        
        matches = 0
        with arcpy.da.UpdateCursor(scheduled_bus_points, ['trip_id', 'time', delay_day, day_status], f"time = time '{current_minute}'") as cursor:
            for row in cursor:
                scheduled_trip_id, scheduled_time, schedule_delay, schedule_status = row
                if scheduled_trip_id in near_trip_ids_time:
                    matches = matches + 1
                    near_time = near_trip_ids_time[scheduled_trip_id]
                    delay_minutes = time_diff_in_minutes(scheduled_time, near_time)
                    row[2] = delay_minutes
                    if delay_minutes > 0:
                        row[3] = 1
                    else:
                        row[3] = 0
                    cursor.updateRow(row)

        # Update current_time
        current_time = current_time + timedelta(minutes = 1)
        print(f"Success, {matches} trips out of {len(near_trip_ids_time)} found. Next start time = {current_time}")

live_workspace = r'C:\Users\BRAUE189\Documents\ArcGIS\Projects\GIS5572\PostgreSQL-35-gis5572(postgres).sde'
table_name = 'gis5572.postgres.live_bus_data'
local_workspace = r'C:\Users\BRAUE189\Documents\ArcGIS\Projects\GIS5572\Default.gdb'
start_time = datetime(2024, 3, 23, 8, 0, 0)
calculate_delay(live_workspace, table_name, local_workspace, start_time)         

Processing points: 228it [06:07,  1.61s/it]﻿


Success, 203 trips out of 219 found. Next start time = 2024-03-23 08:01:00


Processing points: 221it [16:46,  4.55s/it]﻿


Success, 201 trips out of 217 found. Next start time = 2024-03-23 08:02:00


Processing points: 237it [17:53,  4.53s/it]﻿


...

Processing points: 310it [3:04:00, 35.61s/it]﻿


Success, 300 trips out of 305 found. Next start time = 2024-03-23 12:09:00


Processing points: 314it [2:59:24, 34.28s/it]﻿


Success, 284 trips out of 308 found. Next start time = 2024-03-23 12:10:00


ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (MakeQueryLayer).


In [1]:
def create_cloud_class(workspace):
    arcpy.env.workspace = workspace
    spatial_ref = arcpy.SpatialReference(4326)
    out_name = 'bus_delay_20240323'                                   
    geometry_type = "POINT"
    arcpy.management.CreateFeatureclass(arcpy.env.workspace, out_name, geometry_type, spatial_reference = spatial_ref)
    output_class = os.path.join(arcpy.env.workspace, out_name)
    arcpy.management.CopyFeatures(points, output_class)
    
create_cloud_class(r'C:\Users\BRAUE189\Documents\ArcGIS\Projects\GIS5572\PostgreSQL-35-gis5572(postgres).sde', r'C:\Users\BRAUE189\Documents\ArcGIS\Projects\GIS5572\Default.gdb\new_points')